In [ ]:
# default_exp schedule

# Schedules

> When should I prune my network ?

In [ ]:
#export
import numpy as np

In [ ]:
#export

def iterative(start, end, pos, n_steps=3):
    "Perform iterative pruning, and pruning in `n_steps` steps"
    return start + ((end-start)/n_steps)*(np.ceil((pos)*n_steps))
    
def one_shot(start, end, pos): return end

def sched_agp(start, end, pos): return end + start - end * (1 - pos)**3